In [6]:
from cascade.core.db import cursor, connection
from cascade.input_data.db.bundle import _get_bundle_data
import cascade.testing_utilities
from cascade.input_data.configuration.builder import assign_interpolated_covariate_values

ModuleNotFoundError: No module named 'cascade'

In [2]:
execution_context = make_execution_context()

NameError: name 'make_execution_context' is not defined

In [3]:
bundle_id = 173
measurements = _get_bundle_data(execution_context, bundle_id, tier=3)

NameError: name '_get_bundle_data' is not defined

In [4]:
covariate_id = 2
demographics = {}
demographics_default["age_group_ids"] = [
        2, 3, 4, 5, 6,
        7, 8, 9, 10, 11,
        12, 13, 14, 15, 16,
        17, 18, 19, 20, 30,
        31, 32, 235, 27]
demographics_default["year_ids"] = [
        1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
        1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
        2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
        2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 1950, 1951,
        1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
        1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
        1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979]
demographics_default["sex_ids"] = [1, 2]
demographics_default["location_ids"] = [102]
gbd_round_id = 5
covariates = country_covariates(covariate_id, demographics, gbd_round_id)

NameError: name 'demographics_default' is not defined

In [5]:
covariate_column = assign_interpolated_covariate_values(measurements, covariates)

NameError: name 'assign_interpolated_covariate_values' is not defined